In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [3]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

In [5]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/chatbot pdfs")
docs = loader.load()

In [6]:
len(docs)

100

In [7]:
docs[9]

Document(metadata={'source': '/content/drive/MyDrive/chatbot pdfs/Dry-Mouth-Q&A.pdf', 'page': 3}, page_content='Some people feel they have a dry mouth even if their salivary \nglands are working correctly. People with certain disorders, like \nAlzheimer’s disease or those who have suffered a stroke, may \nnot be able to feel wetness in their mouth.\nWhat causes dry mouth?\nPeople get dry mouth when the glands in the mouth that \nmake saliva are not working properly. Because of this, there \nmight not be enough saliva to keep your mouth wet. There \nare several reasons why these glands (called salivary glands) \nmight not work right.\n ● Side effects of some medicines. More than 400 \nmedicines can cause the salivary glands to make less saliva. \nFor example, medicines for high blood pressure  \nand depression often cause dry mouth.\n ● Disease. Some diseases affect the salivary glands. For \nexample, Sjögren’s syndrome, diabetes, HIV/AIDS, and \nHepatitis C can all cause dry mouth.\n ●

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)

In [9]:
len(chunks)


394

In [10]:
chunks[50]

Document(metadata={'source': '/content/drive/MyDrive/chatbot pdfs/Dry-Mouth-Q&A.pdf', 'page': 3}, page_content='swelling and damage in the salivary glands and cause the \nglands to make less saliva or make saliva thicker. As a \nresult, the mouth feels dry.\n ● Nerve Damage. Injury to the head or neck can damage the \nnerves that tell salivary glands to make saliva.')

In [11]:
chunks[51]

Document(metadata={'source': '/content/drive/MyDrive/chatbot pdfs/Dry-Mouth-Q&A.pdf', 'page': 4}, page_content='How will my dentist or physician treat \nmy dry mouth? \nDry mouth treatment will depend on what is causing the \nproblem. \n ● If your dry mouth is caused by medicine, your physician \nmight change your prescription or adjust the dosage.\n ● If your salivary glands are not working right but can still')

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACE_TOKEN', 'your-token-here')

In [13]:
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")


<ipython-input-13-0ff5cf6e8b20>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")


In [15]:
vectorstore = Chroma.from_documents(chunks, embeddings)


In [16]:
query = "Why do we get cavities?"

search_results = vectorstore.similarity_search(query)

In [17]:
search_results


[Document(metadata={'page': 1, 'source': '/content/drive/MyDrive/chatbot pdfs/Dry-Mouth-Q&A.pdf'}, page_content='Dry mouth is not a normal part of aging. So if you think you \nhave dry mouth, see your dentist or physician—there are \nthings you can do to get relief.'),
 Document(metadata={'page': 13, 'source': '/content/drive/MyDrive/chatbot pdfs/Oral Health and Hygiene.pdf'}, page_content='HOW DO YOU GET CAVITIES?\n\uf0a2 Eating sugary foods and liquids frequently \nthroughout the day\n\uf0a2 Not brushing teeth daily\n\uf0a2 Not flossing daily\n\uf0a2 Not using fluoride toothpaste\n\uf0a2 Not visiting the dentist on a regular basis\nNote: Many medicines have sugar too!'),
 Document(metadata={'page': 1, 'source': '/content/drive/MyDrive/chatbot pdfs/Dry-Mouth-Q&A.pdf'}, page_content='to serious health problems. It can also be a sign of certain \ndiseases and conditions.\nWithout enough saliva you can develop tooth decay or other \ninfections in the mouth. You also might not get the nut

In [18]:
retriever = vectorstore.as_retriever(search_kwargs={'k':5})


In [19]:
retriever.get_relevant_documents(query)


<ipython-input-19-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'page': 1, 'source': '/content/drive/MyDrive/chatbot pdfs/Dry-Mouth-Q&A.pdf'}, page_content='Dry mouth is not a normal part of aging. So if you think you \nhave dry mouth, see your dentist or physician—there are \nthings you can do to get relief.'),
 Document(metadata={'page': 13, 'source': '/content/drive/MyDrive/chatbot pdfs/Oral Health and Hygiene.pdf'}, page_content='HOW DO YOU GET CAVITIES?\n\uf0a2 Eating sugary foods and liquids frequently \nthroughout the day\n\uf0a2 Not brushing teeth daily\n\uf0a2 Not flossing daily\n\uf0a2 Not using fluoride toothpaste\n\uf0a2 Not visiting the dentist on a regular basis\nNote: Many medicines have sugar too!'),
 Document(metadata={'page': 1, 'source': '/content/drive/MyDrive/chatbot pdfs/Dry-Mouth-Q&A.pdf'}, page_content='to serious health problems. It can also be a sign of certain \ndiseases and conditions.\nWithout enough saliva you can develop tooth decay or other \ninfections in the mouth. You also might not get the nut

In [20]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="BioMistral/BioMistral-7B-GGUF",
	filename="ggml-model-Q2_K.gguf",
)

llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "Why do we get cavities?"
		}
	]
)


llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--BioMistral--BioMistral-7B-GGUF/snapshots/de8c2dfcead24fd23ccb33f6ca5ff015e9ecdb4b/./ggml-model-Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension

{'id': 'chatcmpl-3936727b-3dcf-4e76-a1af-15ab50789589',
 'object': 'chat.completion',
 'created': 1731207340,
 'model': '/root/.cache/huggingface/hub/models--BioMistral--BioMistral-7B-GGUF/snapshots/de8c2dfcead24fd23ccb33f6ca5ff015e9ecdb4b/./ggml-model-Q2_K.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': " Well, cavities are caused by certain types of bacteria that live in our mouths. When we eat, the bacteria use the sugars in the food to make energy. This process also makes acids. These acids create a special environment in your mouth that can cause tooth decay. That's why it's important to clean your teeth every day, even if you have a cavity and need to go to the dentist. If you don't clean your teeth, the acids can still hurt your teeth even if you're getting a new tooth."},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 15, 'completion_tokens': 116, 'total_tokens': 131}}

In [22]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [23]:
template = """
<|context|>
You are a Medical Assistant specializing in dental health. Follow instructions and generate accurate responses based on the query and context provided.
Please be truthful and give direct answers.

<|user|>
{query}

<|assistant|>
"""

In [24]:
prompt = ChatPromptTemplate.from_template(template)

# Define the RAG chain
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
# Step 1: Retrieve relevant documents using retriever
query = "What are the symptoms of gum disease?"
retrieved_docs = retriever.get_relevant_documents(query)

# Step 2: Concatenate document content to form the context
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# Step 3: Define the input for the Llama model with context and query
rag_input = {"context": context, "query": query}

In [32]:
response = prompt.format(**rag_input)  # Fill prompt with context and query
generated_response = llm.create_chat_completion(messages=[{"role": "user", "content": response}])

# Print the response
answer = generated_response['choices'][0]['message']['content']

print("Answer:", answer)

Llama.generate: 78 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    8467.78 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    58 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   43584.15 ms /    59 tokens


Answer:  The symptoms of gum disease include red or swollen gums, bleeding when you brush your teeth, loose-fitting tooth, bad breath or a bad taste in your mouth, and pus from the tooth. Other signs are tooth decay and tooth sensitivity to touch or bite.


In [33]:
while True:
    # Get user query
    user_query = input("Ask a dental health question (or type 'exit' to quit): ")

    if user_query.lower() == "exit":
        print("Exiting chat. Goodbye!")
        break

    # Step 2: Retrieve relevant documents based on user query
    retrieved_docs = retriever.get_relevant_documents(user_query)

    # Step 3: Concatenate retrieved document content into context
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Step 4: Prepare prompt with context and user query
    rag_input = {"context": context, "query": user_query}
    formatted_prompt = prompt.format(**rag_input)

    # Step 5: Generate the assistant's response
    response = llm.create_chat_completion(messages=[{"role": "user", "content": formatted_prompt}])

    # Extract the assistant's answer
    answer = response['choices'][0]['message']['content']

    # Print the response
    print("Answer:", answer)

Ask a dental health question (or type 'exit' to quit): what is the reason for cavities?


Llama.generate: 57 prefix-match hit, remaining 21 prompt tokens to eval
llama_perf_context_print:        load time =    8467.78 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    21 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    63 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   50742.23 ms /    84 tokens


Answer:  Cavities occur when there is an imbalance between the mineralization of tooth surfaces and the deposition of acids in the mouth. Poor oral hygiene habits such as not brushing twice a day, not using fluoride, and consuming many sweets can also be a reason for cavities.
Ask a dental health question (or type 'exit' to quit): why is my gum bleeding?


Llama.generate: 57 prefix-match hit, remaining 20 prompt tokens to eval
llama_perf_context_print:        load time =    8467.78 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    20 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    66 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   54866.61 ms /    86 tokens


Answer:  A bleeding gum is often a sign of an active infection. It can be caused by a variety of factors, including a wound in the mouth, gum disease, or a build-up of plaque, also known as gingivitis. It's essential to find the source of the bleeding and treat it immediately.
Ask a dental health question (or type 'exit' to quit): What are the common causes of dry mouth in older adults?


Llama.generate: 57 prefix-match hit, remaining 25 prompt tokens to eval
llama_perf_context_print:        load time =    8467.78 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    25 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    80 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   65535.20 ms /   105 tokens


Answer:  Some common causes of dry mouth in older adults include: 1. Decreased salivary flow due to medications, 2. Diabetes, 3. Autoimmune diseases, 4. Surgical treatment, 5. Smoking and alcohol use, 6. Certain medications, 7. Radiation therapy and 8. Cancer of the head and neck.
Ask a dental health question (or type 'exit' to quit): Why is it important to catch mouth and throat cancer early?


Llama.generate: 57 prefix-match hit, remaining 25 prompt tokens to eval
llama_perf_context_print:        load time =    8467.78 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    25 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    59 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   51402.44 ms /    84 tokens


Answer:  In order to treat the infection effectively, it is crucial to diagnose mouth and throat cancer as early as possible. Early detection of these cancers can prevent their spread to other organs and may improve the chances of survival. Early detection can also enable patients to undergo less invasive treatments.
Ask a dental health question (or type 'exit' to quit): exit
Exiting chat. Goodbye!
